# Cryptopunks: obtener datos de un cryptopunk

In [1]:
import pandas as pd
import numpy as np

from datetime import datetime

import requests
from bs4 import BeautifulSoup

import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

from selenium.webdriver.common.keys import Keys
from scrapy import Selector
import math

import mysql.connector

import re

## Establecer conexión con la BD

In [2]:
# Connect to the new database
mydb = mysql.connector.connect(
  host="localhost",
  user="cecilia",
  password="cecilia",
  database="cryptopunks"
)

mycursor = mydb.cursor()



## Acceso a la página de un cryptopunk

In [3]:
# Define the cryptopunt to scrap
#nro_cp = 8348
#nro_cp = 5214
nro_cp = 4347

url = 'https://www.larvalabs.com/cryptopunks/details/'+str(nro_cp)



In [4]:
# Use BeautifulSoup por webscrapping
html=requests.get(url)
soup = BeautifulSoup(html.content, 'html.parser')
#print(soup)

## Obtener la información del cryptopunk

In [5]:
def obtener_cryptopunk(soup, mydb):
    # Obtener el tipo de cryptopunk
    h4 = soup.find_all('h4')
    tipo_cp = h4[0].text.split()[3]
    print(tipo_cp)
    
    # Obtener el nombre del owner, y saber si está a la venta y su precio
    varios =soup.find_all('div', class_="col-md-10 col-md-offset-1")

    for linea in varios:
        texto = linea.text
        # Busco la línea que comienza con ...
        if texto.strip().startswith('Current Market Status') and texto.find('currently')>0:
            # Extraigo el nombre del dueño
            if texto.find("This punk is currently owned by address")>0:
                owner = texto.split('address ')[1].split('.',1)[0]
                print('Owner:',owner)
        
            # Averiguo si está en venta
            precio_venta = 0
            if texto.find("This punk is currently for sale by owner")>0:
                sale = True
                precio_venta = texto.split('by owner for ')[1].split('Ξ',1)[0]
                print('Precio de venta', precio_venta)
            else:
                sale = False # Dice 'This punk has not been listed for sale by its owner'
            print('For sale',sale)
        
            # Averiguo si hay ofertas
            oferta = 0
            if texto.find("There is a bid of")>0:
                bids = True
                oferta = texto.split('is a bid of ')[1].split('Ξ',1)[0]
                print('Valor ofrecido', oferta)
             
            else:
                bids = False  # Dice "There are currently no bids on this punk."
            print('Bids:',bids)

    # Obtain the typeID
    mycursor.execute("SELECT tyID FROM types WHERE at_type = %s", (tipo_cp.strip(),))
    id_tipo = mycursor.fetchone()[0]

    # Insert cryptopunk data into the table
    sql = "INSERT INTO cryptopunks (cpID, owner, tyID, for_sale, bids) VALUES (%s, %s, %s, %s, %s )"

    mycursor.execute(sql, (nro_cp, owner, id_tipo, sale, bids ))

    
    mydb.commit()        

## Obtener la lista de transacciones realizadas

In [6]:
def obtener_transacciones(soup, mydb):
    
    def obtener_montos(str1):
        salida = re.search("(((\(\$)\s*\d+)([,]\d+){0,1}([,]\d+){0,1}([.]?\d{2})?)[MK]?\)", str1)
        con_coma = salida[0][2:-1]
        sin_coma = con_coma.replace(',','')
        if sin_coma.find('M')>0:
            sin_coma = sin_coma[0:-1]
            monto_dols = float(sin_coma)
            monto_dols = monto_dols * 1000000
        elif sin_coma.find('K')>0:
            sin_coma = sin_coma[0:-1]
            monto_dols = float(sin_coma)
            monto_dols = monto_dols * 1000
        else: 
            monto_dols = float(sin_coma)
        con_E = re.search("(\d+([,]\d+)?([.]\d{2})?)?\Ξ", str1)
        monto = float(con_E[1].replace(',',''))
        return monto, monto_dols
        
    text1 = soup.find_all('a', text = '')
    #print(text1)
    
    transacciones =[]
    
    transac =soup.find('table', class_="table")
    #print(transac)

    for tr in transac.find_all('tr'):
        lista = []
        for td in tr.find_all('td'):
            lista.append(td.text.strip())
        transacciones.append(lista)
    transacciones = transacciones[1:]
    print(transacciones)
    
  
    # Insert data into the table
    sql = "INSERT INTO transactions (cpID, tr_type, tr_from, tr_to, amount, amount_dols, txn) VALUES (%s, %s, %s, %s, %s, %s, %s)"

    for tr in transacciones:
        # si la transaccion no esta vacia
        if len(tr)>0:
            # Hay un monto de transacción?
            monto = 0
            monto_dols = 0
            if len(tr[3])>0:
                monto, monto_dols = obtener_montos(tr[3])
            
            fecha = datetime.strptime(tr[4], "%b %d, %Y")
            
    
            mycursor.execute(sql, (nro_cp, tr[0],  tr[1], tr[2], monto, monto_dols, fecha))

    
    mydb.commit()

## Obtener atributos del cryptopunk

In [7]:
def obtener_atributos(soup, mydb):
    cod =soup.find_all('div', class_="col-md-4")
    textos = []
    for c in cod:
        textos.append(c.find('a').text)
    
    atributos = textos[1:]
    print('Atributos:', atributos)
    
    # Insert data into the table attributes
    sql = "INSERT INTO cp_attributes (cpID, atID) VALUES (%s, %s)"

    for atr in atributos:
        # Obtain the typeID
        mycursor.execute("SELECT atID FROM attributes WHERE attribute = %s", (atr.strip(),))
        id_atr = mycursor.fetchone()[0]
        
        mycursor.execute(sql, (nro_cp,id_atr))


    mydb.commit()

In [8]:
obtener_cryptopunk(soup, mydb)
obtener_transacciones(soup, mydb)
obtener_atributos(soup, mydb)

Female
Owner: 0xc3c14f
Precio de venta 288
For sale True
Bids: False
[['Offered', '', '', '288Ξ ($1.26M)', 'Oct 30, 2021'], ['Offered', '', '', '888.80Ξ ($3.45M)', 'Sep 07, 2021'], ['Offered', '', '', '400Ξ ($1.52M)', 'Sep 02, 2021'], ['Sold', '0x50dd57', '0xc3c14f', '222Ξ ($703,216)', 'Aug 29, 2021'], ['Offered', '', '', '222Ξ ($704,171)', 'Aug 24, 2021'], ['Offer Withdrawn', '', '', '', 'Aug 23, 2021'], ['Offered', '', '', '130Ξ ($396,741)', 'Aug 19, 2021'], ['Transfer', '0xb1adce', '0x50dd57', '', 'Mar 16, 2021'], ['Sold', '0xc352b5', '0xb1adce', '45Ξ ($87,922)', 'Feb 21, 2021'], ['Offered', '', '', '45Ξ ($85,990)', 'Feb 19, 2021'], ['Bid Withdrawn', 'Pranksy', '', '1Ξ ($136)', 'Jan 05, 2020'], ['Bid', 'Pranksy', '', '1Ξ ($132)', 'Jan 01, 2020'], ['Bid', '0x717403', '', '0.05Ξ ($13)', 'Jun 26, 2017'], ['Claimed', '', '0xc352b5', '', 'Jun 23, 2017']]
Atributos: ['Pink With Hat', 'Nerd Glasses']


In [ ]:
        str1 = 
        salida = re.search("(((\(\$)\s*\d+)([,]\d+){0,1}([,]\d+){0,1}([.]?\d{2})?)[MK]?\)", str1)
        con_coma = salida[0][2:-1]
        sin_coma = con_coma.replace(',','')
        if sin_coma.find('M')>0:
            sin_coma = sin_coma[0:-1]
            monto_dols = float(sin_coma)
            monto_dols = monto_dols * 1000000
        elif sin_coma.find('K')>0:
            sin_coma = sin_coma[0:-1]
            monto_dols = float(sin_coma)
            monto_dols = monto_dols * 1000
        else: 
            monto_dols = float(sin_coma)
        con_E = re.search("(\d+([,]\d+)?([.]\d{2})?)?\Ξ", str1)
        monto = float(con_E[1].replace(',',''))
        return monto, monto_dols
